In [ ]:
! pip install pandas
! pip install num2words
! pip install numpy
! pip install ollama

In [ ]:
import os, re, sys
import pandas as pd
import numpy as np

df=pd.read_csv(os.path.join(os.getcwd(),'input/Suchanfragen.csv')) 
df.head()

In [ ]:
df_qrs = df[['Häufigste Suchanfragen']]
df_qrs = df_qrs.rename(columns={'Häufigste Suchanfragen': 'queries'})
df_qrs

In [ ]:
import ollama

def generate_embeddings(text, model="mxbai-embed-large"):
    return ollama.embeddings(model=model, prompt=text)["embedding"]

df_qrs['embeddings'] = df_qrs["queries"].apply(lambda x : generate_embeddings (x, model = 'mxbai-embed-large'))
# df_qrs.head()

In [ ]:
input = "was ist analytics?"

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="mxbai-embed-large"):
    return ollama.embed(input = [text], model=model)



def search_docs(df, user_query, top_n=1, to_print=True):
    embedding = get_embedding(
        user_query,
        model="mxbai-embed-large" 
    )['embeddings'][0]
    # print(embedding)
    df["similarities"] = df.embeddings.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res   

res = search_docs(df_qrs, input, top_n=10)['queries']
res.reset_index(drop=True, inplace=True)
# print(res)

In [ ]:
output = ollama.generate(
    model='stablelm2',
    prompt=f"Based on the query: {input}, please provide a detailed response based on the following data: {res[1]}",
)
print(output['response'])